<a href="https://colab.research.google.com/github/JuGP/CrawlerSyllabicDivision/blob/master/CrawlerSyllabicDivision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Packages

In [ ]:
import re
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

#Constants

In [ ]:
LETTERS = "abcdefghijklmnopqrstuvwxyz"
URL = "http://www.portaldalinguaportuguesa.org/index.php?action=syllables&act=list&letter={}&start={}"

DATASET_FILE = "letter_{}.csv"
ALL_DATASET_FILE = "letters.csv"

WORD = "Word"
CLASS = "Class"
PRONUNCIATION = "Pronunciation"

#Crawler

In [ ]:
def get_page_elements(page):
    content = BeautifulSoup(page.content, "html5lib")
    table = content.find('table', attrs={"id":"rollovertable"})
    elements = table.findAll('tr', attrs={"class":"even"})
    return elements

def extract_element_information(element):
    word = re.search("(.*) \((.*)\)", element.find('td', attrs={"title":"Palavra"}).text)
    pronunciation = element.find('td', attrs={"title":"pron�ncia"}).text.strip()
    serie = pd.Series([word.group(1), word.group(2), pronunciation], index=[WORD, CLASS, PRONUNCIATION], name=letter)
    return serie

for letter in LETTERS:
    df = pd.DataFrame(columns=[WORD, CLASS, PRONUNCIATION])
    start = 0
    while True:
        print("letter: {} start: {}".format(letter,start))
        page = requests.get(URL.format(letter,start), timeout=5)
        elements = get_page_elements(page)

        if len(elements) <= 1:
            break
        else:
            for element in elements:
                serie = extract_element_information(element)
                df = df.append(serie)
            start += 100
        time.sleep(3)
    df.to_csv(DATASET_FILE.format(letter))

In [ ]:
files = [DATASET_FILE.format(letter) for letter in LETTERS]
df = pd.concat([pd.read_csv(file, index_col=0) for file in files])
df.to_csv(ALL_DATASET_FILE)